In [1]:
%pwd

'd:\\Projects\\uob-project\\src'

In [2]:
%cd ..

d:\Projects\uob-project


In [41]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from src.main_model import BitcoinRNN


NEWS_PER_DAY = 3
TRAIN_FACTOR = 0.8
LOOK_BACK = 30
PRED_COL = "change"
DATA_SENTIMENTAL_PATH = "data/processed/sentimental_3_per_day.csv"
DATA_BITCOIN_PATH = "data/processed/bitcoin.csv"

In [14]:
sentimental = pd.read_csv(DATA_SENTIMENTAL_PATH, index_col=0)
bitcoin = pd.read_csv(DATA_BITCOIN_PATH, index_col=0)

In [ ]:
# Process sentimental data
sentimental["similarity"] = sentimental["similarity"].apply(eval)
sentimental["search"] = sentimental["search"].apply(eval)

In [45]:
similarities = []
searches = []

for i in range(NEWS_PER_DAY):
    every_nth = sentimental.iloc[i::NEWS_PER_DAY]
    similarities.append(every_nth["similarity"])
    searches.append(every_nth["search"])

x_similarities = np.array(similarities)
x_searches = np.array(searches)
x_bitcoin = np.array(bitcoin.drop([PRED_COL], axis=1).values)
y_bitcoin = np.array(bitcoin[PRED_COL].values)

In [46]:
num_train = int(len(y_bitcoin) * TRAIN_FACTOR)

x_train = {
    "x_bitcoin": x_bitcoin[:num_train],
    "x_similarities": x_similarities[:, :num_train],
    "x_searches": x_searches[:, :num_train]
}

x_test = {
    "x_bitcoin": x_bitcoin[num_train:],
    "x_similarities": x_similarities[:, num_train:],
    "x_searches": x_searches[:, num_train:]
}

y_train = y_bitcoin[:num_train]
y_test = y_bitcoin[num_train:]

In [47]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(x_train["x_bitcoin"])

x_train["x_bitcoin"] = scaler.transform(x_train["x_bitcoin"])
x_test["x_bitcoin"] = scaler.transform(x_test["x_bitcoin"])

In [48]:
def create_sequences(x, y, look_back=LOOK_BACK):
    x_bitcoin_sequences = []
    x_similarity_sequences = []
    x_search_sequences = []
    ys = []

    for i in range(len(y) - look_back):
        x_bitcoin_sequences.append(x["x_bitcoin"][i:i+look_back])
        x_similarity_sequences.append(x["x_similarities"][:, i:i+look_back])
        x_search_sequences.append(x["x_searches"][:, i:i+look_back])
        ys.append(y[i+look_back])
    
    x = {
        "x_bitcoin": tf.constant(x_bitcoin_sequences, dtype=tf.float32),
        "x_similarities": tf.constant(x_similarity_sequences, dtype=tf.float32),
        "x_searches": tf.constant(x_search_sequences, dtype=tf.float32)
    }

    y = tf.constant(ys)

    return x, y

x_train, y_train = create_sequences(x_train, y_train)
x_test, y_test = create_sequences(x_test, y_test)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [39]:
print("Similarites shape:", x_train["x_similarities"].shape)
print("Searches shape:", x_train["x_searches"].shape)
print("Bitcoin shape:", x_train["x_bitcoin"].shape)
print("Predict shape:", y_train.shape)

Similarites shape: (3, 876, 4096)
Searches shape: (3, 876, 4096)
Bitcoin shape: (876, 6)
Predict shape: (876,)


In [38]:
model_kwargs = {
    "in_shape_main": (30, 6),
    "in_shape_sent": (4096,),
    "use_sentimental": True,
    "architecture": "SIMPLE",
    "out_neurons_sent": 10,
    "sent_arch": "DENSE",
    "look_back": LOOK_BACK
}

compile_kwargs = {
    "loss": "mean_squared_error",
    "optimizer": tf.keras.optimizers.Adam(learning_rate=1e-3),
    "metrics": tf.keras.metrics.RootMeanSquaredError()
}

model = BitcoinRNN(**model_kwargs)
model.compile(**compile_kwargs)

history= model.fit(x_train, y_train, validation_split=0.30, epochs=30, batch_size=16, shuffle=False)

# Plot training
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

<tf.Tensor: shape=(876, 6), dtype=float64, numpy=
array([[0.0057439 , 0.00688479, 0.00537604, 0.007388  , 0.        ,
        0.00724925],
       [0.0074488 , 0.00846479, 0.00724925, 0.00904994, 0.00265567,
        0.00739909],
       [0.0088099 , 0.00826464, 0.00739909, 0.00727522, 0.00059426,
        0.00667875],
       ...,
       [0.5676767 , 0.56744831, 0.47326233, 0.52194726, 0.21387439,
        0.52977132],
       [0.52058798, 0.59261178, 0.52977132, 0.58742376, 0.18182805,
        0.56429051],
       [0.58870601, 0.59165485, 0.56429051, 0.58236988, 0.14967216,
        0.58680756]])>